In [1]:
# Importation of necessary packages/modules

import numpy as np 
import pandas as pd 

import tensorflow as tf
from tensorflow import keras

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

#Importation of dataset for training/testing data
import os
for dirname, _, filenames in os.walk('/kaggle/input/disease-prediction-using-machine-learning'):
    for filename in filenames:
        if filename == 'Training.csv':
            train_filepath = os.path.join(dirname, filename)
            training_data = pd.read_csv(train_filepath)
        elif filename == 'Testing.csv':
            test_filepath = os.path.join(dirname, filename)
            test_data = pd.read_csv(test_filepath)

#Check content of data variables
print("Training Data:")
training_data.info()
print("\nTest Data:")
test_data.info()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Training Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 134 entries, itching to Unnamed: 133
dtypes: float64(1), int64(132), object(1)
memory usage: 5.0+ MB

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 43.8+ KB


**Data Preprocessing**

In [2]:
# Check for columns with null values
training_data.isnull().sum()

itching                    0
skin_rash                  0
nodal_skin_eruptions       0
continuous_sneezing        0
shivering                  0
                        ... 
blister                    0
red_sore_around_nose       0
yellow_crust_ooze          0
prognosis                  0
Unnamed: 133            4920
Length: 134, dtype: int64

In [3]:
# Preprocess categorical data in prognosis column into numerical values
dataframes = [training_data, test_data]

label_encoder = preprocessing.LabelEncoder()
for dataframe in dataframes:
    dataframe['prognosis'] = label_encoder.fit_transform(dataframe['prognosis'])

In [4]:
# Convert DataFrame to Numpy Arrays
training_numpy = training_data.to_numpy()
test_numpy = test_data.to_numpy()

In [5]:
# Separate features to target variables for both train/test data
x_train = training_numpy[:,0:132]
x_test = test_numpy[:,0:132]

y_train = training_numpy[:,132]
y_test = test_numpy[:,132]

In [6]:
# Scale data for effective NN training
data_scaler = StandardScaler().fit(x_train)

x_train = data_scaler.transform(x_train)
x_test = data_scaler.transform(x_test)